# Production-Ready Agent Workflow with Flotorch and LangChain

This notebook demonstrates a comprehensive, production-style workflow. It integrates all the advanced features we've explored into a single, robust application:
- **Remote Agent Configuration**: Uses `FlotorchLangChainAgent` to load agent logic from the Flotorch UI.
- **Custom Local Tools**: Defines and attaches a custom tool to the agent at runtime.
- **Dual Memory System**: Implements both short-term session memory (`FlotorchLangChainSession`) and long-term memory (`FlotorchLangChainMemory`).

## 1. Setup and Imports

The following cells install dependencies, set API credentials, and import all necessary components for handling tools, memory, and remote agent management.

In [ ]:
# install flotorch langchain package
%pip install  flotorch[langchain]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "https://qa-gateway.flotorch.cloud"
MEMORY_PROVIDER = "<flotorch memory provider>"
AGENT_NAME = "<flotorch agent name>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"

In [ ]:
# Import LangChain and related modules
from flotorch.langchain.agent import FlotorchLangChainAgent 
from flotorch.langchain.session import FlotorchLangChainSession
from flotorch.langchain.memory import FlotorchLangChainMemory
from langchain.agents import AgentExecutor
from langchain.memory import CombinedMemory

from langchain_core.tools import tool

print("Imported necessary libraries successfully")

## 2. Custom Tool Definition

Here, we define custom tools locally. These tools can be dynamically attached to our agent at runtime, demonstrating how to extend a centrally-managed agent with new capabilities.

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


@tool
def weather(city: str) -> str:
    """Get weather for a city. Available cities: New York, London, Tokyo, Paris."""
    city = city.lower().strip()

    weather_info = {
        "new york": "Sunny, 72°F, Humidity: 45%",
        "london": "Cloudy, 15°C, Humidity: 70%",
        "tokyo": "Rainy, 25°C, Humidity: 80%",
        "paris": "Partly cloudy, 18°C, Humidity: 60%"
    }

    if city in weather_info:
        return f"Weather in {city.title()}: {weather_info[city]}"
    else:
        return f"I don't have weather data for '{city}'. Try: New York, London, Tokyo, or Paris."

tools = [analyze_text, weather]

print("Custom tool defined successfully.")

## 3. Memory Configuration

We configure the full dual-memory system: `FlotorchLangChainSession` for conversational context and `FlotorchLangChainMemory` for persistent, long-term knowledge. These are then wrapped in LangChain's `CombinedMemory`.

In [ ]:
short_term_memory = FlotorchLangChainSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

external_memory = FlotorchLangChainMemory(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_ID,
)

memory_services  = CombinedMemory(memories = [short_term_memory,external_memory])
print("Initialized short-term and external Memories")

## 4. Agent Initialization with Custom Tools

We initialize the `FlotorchLangChainAgent` client, loading our agent's configuration from the Flotorch UI. Crucially, we also pass our locally defined tools into the `custom_tools` parameter, augmenting the agent's capabilities.

In [ ]:
flotorch_client = FlotorchLangChainAgent(
    api_key=FLOTORCH_API_KEY,
    enable_memory = True,
    base_url=FLOTORCH_BASE_URL,
    agent_name=AGENT_NAME,
    custom_tools = tools
)

agent = flotorch_client.get_agent()
all_tools = flotorch_client.get_tools()

print("Fetched agent and task from Flotorch console")

## 5. Assembling the Complete Agent Executor

Finally, we assemble the `AgentExecutor`, bringing together the remotely-configured agent, its combined tools, and the `CombinedMemory` module. This creates a fully-featured, production-ready agent.

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=all_tools,
    memory = memory_services,
    verbose=False,
    handle_parsing_errors=True
)

## 6. Interactive Demonstration

Engage with the agent in this interactive loop to test its full range of capabilities. Try asking it to analyze text (to test the tool) and then ask it to recall information from the conversation. Type 'exit' to end the session.

In [ ]:
while True:
    
    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    print(f"User: {user_query}")
    response = agent_executor.invoke({"input":user_query})
    print(f"Assistant: {response['output']}")

## Summary

This notebook served as a **capstone project**, successfully integrating all previously demonstrated concepts into a single, comprehensive, and production-ready workflow.  
We built an advanced AI agent that combines the power of **remote configuration**, **local custom tools**, and a **full dual-memory system**.

### Key Achievements

- **Hybrid Configuration** 
  Initialized an agent using `FlotorchLangChainAgent` to load its base configuration from the Flotorch UI, then augmented it at runtime with locally defined custom tools.

- **Full Memory Stack** 
  Equipped the agent with both `FlotorchLangChainSession` for conversational flow and `FlotorchLangChainMemory` for long-term knowledge persistence, managed by `CombinedMemory`.

- **End-to-End Demonstration** 
  Showcased the agent's ability to seamlessly switch between using its custom tools for specific tasks and accessing its memory to provide contextually rich, intelligent responses.

- **Robust Architecture** 
  Delivered a complete workflow that serves as a powerful and flexible blueprint for building sophisticated, real-world AI applications.
